In [1]:
import pandas as pd
import sqlite3
import numpy as np
import sqlalchemy

In [28]:
#source: https://www.nature.com/articles/s41562-020-0909-7

#filter low level changes and duplicate descriptions
context= pd.read_csv("./coronanet_release_allvars.csv", low_memory=False)
context = context[context['entry_type']!="update"]
context = context[context['correct_type']!="correction"]
context = context[context['init_country_level']=="National"] #ignore regional
context = context[~context['country'].str.contains(",")] #ignore measures implemented by multiple countries at once


context.drop_duplicates(subset ="description", keep = False, inplace = True)

#drop not needed cols
context = context.drop(['record_id', 'policy_id', 'date_end', 'date_announced', 'enforcer', 'dist_index_high_est', 'dist_index_med_est', 'dist_index_low_est', 'dist_index_country_rank', 'institution_status', 'target_country', 'target_geog_level', 'target_region', 'target_province', 'target_city', 'target_other', 'target_who_what', 'target_direction', 'link', 'date_updated', 'recorded_date', 'Rank_FP', 'Score_FP', 'muni_IDC', 'dispersive_IDC', 'constraining_IDC', 'sfi_SFI', 'ti_cpi_TI', 'pop_WDI_PW', 'gdp_WDI_PW', 'growth_WDI_PW', 'lnpop_WDI_PW', 'lngdp_WDI_PW', 'disap_FA', 'polpris_FA', 'latentmean_FA', 'transparencyindex_HR', 'EmigrantStock_EMS', 'type_sub_cat'], axis = 1)
context = context.drop(['tests_units', 'total_vaccinations', 'daily_vaccinations', 'total_vaccinations_per_hundred', 'daily_vaccinations_per_million','confirmed_cases', 'deaths', 'recovered', 'new_tests', 'total_tests', 'total_tests_per_thousand'], axis = 1)
context = context.drop(['province', 'ISO_L2','ISO_A3','ISO_A2', 'city'], axis = 1)
context = context.drop(['update_level', 'correct_type', 'travel_mechanism', 'compliance', 'entry_type', 'update_type', 'type_text', 'init_country_level', 'domestic_policy'], axis = 1)


#bring to shape (similar to memories)
context = context.rename(columns={"date_start": "date", "type": "category", "description": "comment"})
context['date'] = pd.to_datetime(context['date'], format='%Y/%m/%d')
context['date'] = context['date'].dt.date
context["weight"] = 8

#remove "other" from hashtags
context.loc[context["category"].str.contains("Other")]["category"] = np.NaN

#remove data that are too early
context = context[context['date'] > np.datetime64('2020-01-20')]

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [33]:
#SQLTE
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
context.to_sql(con=sqlite, name='contexts', index=True, if_exists='replace',dtype={"date": str(sqlalchemy.types.DateTime())})

In [32]:


context['category'] = context['category'].replace({'Other Policy Not Listed Above': ''})

#context.loc[context["category"].str.contains("Other")]